In [1]:
import pandas as pd
import numpy as np
import folium
from matplotlib import pyplot as plt
from io import BytesIO
import base64

intersection_data = pd.read_csv('D:/Dr. Shamshiripour/PAG_TMC/TmcIntersection.CSV')
intersection_data = intersection_data[['IntersectionId', 'Latitude', 'Longitude']]
count_data = pd.read_csv('D:/Dr. Shamshiripour/PAG_TMC/TmcCount.CSV')
count_data = count_data[['CountId', 'IntersectionId', 'StartDate']]
count_data['IntersectionId'] = count_data['IntersectionId'].astype(int)
count_data['StartDate'] = (count_data['StartDate'] / 1e9).astype(int).astype(str)
count_data['StartDate'] = pd.to_datetime(count_data['StartDate'], format='%Y%m%d', errors = 'coerce')
count_data['Year'] = count_data['StartDate'].dt.year
count_data['DayOfWeek'] = count_data['StartDate'].dt.day_name()
count_data.dropna(subset = ['IntersectionId'], inplace = True)
most_recent_years = count_data.groupby('IntersectionId')['Year'].max()
count_data = count_data[count_data['Year'] == count_data['IntersectionId'].map(most_recent_years)] 
count_data = count_data[~count_data['DayOfWeek'].isin(['Saturday', 'Sunday'])]

am_peak_start = pd.to_datetime('06:30:00', format = '%H:%M:%S').time()
am_peak_end = pd.to_datetime('08:30:00', format = '%H:%M:%S').time()
pm_peak_start = pd.to_datetime('16:00:00', format = '%H:%M:%S').time()
pm_peak_end = pd.to_datetime('18:00:00', format = '%H:%M:%S').time()

chunks = []
for chunk in pd.read_csv('D:/Dr. Shamshiripour/PAG_TMC/TmcCountData.CSV', chunksize = 50000,
                         usecols = ['CountId', 'ClassificationId', 'StartTime',
                                    'HardLeftMovement', 'LeftMovement', 'BearLeftMovement', 
                                    'ThruOrPedMovement', 'BearRightMovement', 'RightMovement', 
                                    'HardRightMovement', 'UTurnMovement']):
    chunk['StartTime'] = pd.to_datetime(chunk['StartTime'], format = '%H:%M:%S').dt.time
    chunk = chunk[((chunk['StartTime'] >= am_peak_start) & (chunk['StartTime'] < am_peak_end)) | 
                  ((chunk['StartTime'] >= pm_peak_start) & (chunk['StartTime'] < pm_peak_end))]
    chunk = chunk.merge(count_data, on = 'CountId', how = 'inner')
    chunk = chunk.merge(intersection_data, on = 'IntersectionId', how = 'inner')
    chunk['TotalCount'] = chunk[['HardLeftMovement', 'LeftMovement', 'BearLeftMovement', 
                                 'ThruOrPedMovement', 'BearRightMovement', 'RightMovement', 
                                 'HardRightMovement', 'UTurnMovement']].sum(axis = 1)
    chunk = chunk[chunk['ClassificationId'] == 1]
    chunks.append(chunk)
    del chunk
cars = pd.concat(chunks, ignore_index = True)
car_summary = cars.groupby(['IntersectionId', 'ClassificationId', 'Latitude', 
                            'Longitude', 'StartDate', 'Year'])['TotalCount'].sum().reset_index()
car_summary = car_summary.groupby(['IntersectionId', 'ClassificationId', 'Latitude', 
                                   'Longitude', 'Year']).agg(AverageWeekdayVolume = ('TotalCount', 'mean')).reset_index()

In [2]:
def calculate_radius(count, max_count):
    return (count / max_count) * 30
max_count = car_summary['AverageWeekdayVolume'].max()
m1 = folium.Map(location = [32.2226, -110.9747], zoom_start = 10, tiles = 'cartodbpositron')
for index, row in car_summary.iterrows():
    if row['AverageWeekdayVolume'] > 0:
        folium.CircleMarker(
            location = (row['Latitude'], row['Longitude']),
            radius = calculate_radius(row['AverageWeekdayVolume'], max_count),
            color = '#89d1ff',
            fill = True,
            fill_opacity = 0.5,
            popup = f"<b>Intersection ID:</b> {row['IntersectionId'].astype(int)}<br>" + 
                    f"<b>Year:</b> {row['Year'].astype(int)}<br>" + 
                    f"<b>Average Weekday Car Count:</b> {row['AverageWeekdayVolume'].round(2)}"
        ).add_to(m1)
legend_html = '''
<div style = 'position: fixed; bottom: 50px; left: 50px; width: 250px; height: auto; border: 2px solid grey; 
              z-index: 9999; font-size: 14px; background-color: white; padding: 10px;'>
<b>Time</b><br>
AM: 6:30 - 8:30<br>
PM: 4:00 - 6:00<br>
<b>Average Weekday Car Count</b><br>
'''
for count in [5000, 10000, 15000, 20000]:
    radius = calculate_radius(count, max_count) * 1.5
    legend_html += f'''
    <div style = 'margin: 5px 0;'>
        {count}
        <div style = 'display: inline-block; width: {radius:.2f}px; height: {radius:.2f}px; 
                      background-color: #89d1ff; border-radius: 100%; margin-left: 10px;'></div>
    </div>
    '''
legend_html += '</div>'
m1.get_root().html.add_child(folium.Element(legend_html))
m1.save('car_peak.html')
print('Map has been created and saved to car_peak.html')

Map has been created and saved to car_peak.html


In [3]:
chunks = []
for chunk in pd.read_csv('D:/Dr. Shamshiripour/PAG_TMC/TmcCountData.CSV', chunksize = 50000,
                         usecols = ['CountId', 'ClassificationId', 'StartTime',
                                    'HardLeftMovement', 'LeftMovement', 'BearLeftMovement', 
                                    'ThruOrPedMovement', 'BearRightMovement', 'RightMovement', 
                                    'HardRightMovement', 'UTurnMovement']):
    chunk['StartTime'] = pd.to_datetime(chunk['StartTime'], format = '%H:%M:%S').dt.time
    chunk = chunk[((chunk['StartTime'] >= am_peak_start) & (chunk['StartTime'] < am_peak_end)) | 
                  ((chunk['StartTime'] >= pm_peak_start) & (chunk['StartTime'] < pm_peak_end))]
    chunk = chunk.merge(count_data, on = 'CountId', how = 'inner')
    chunk = chunk.merge(intersection_data, on = 'IntersectionId', how = 'inner')
    chunk['TotalCount'] = chunk[['HardLeftMovement', 'LeftMovement', 'BearLeftMovement', 
                                 'ThruOrPedMovement', 'BearRightMovement', 'RightMovement', 
                                 'HardRightMovement', 'UTurnMovement']].sum(axis = 1)
    chunk = chunk[chunk['ClassificationId'] == 2]
    chunks.append(chunk)
    del chunk
trucks = pd.concat(chunks, ignore_index = True)
truck_summary = trucks.groupby(['IntersectionId', 'ClassificationId', 'Latitude', 
                                'Longitude', 'StartDate', 'Year'])['TotalCount'].sum().reset_index()
truck_summary = truck_summary.groupby(['IntersectionId', 'ClassificationId', 'Latitude', 
                                       'Longitude', 'Year']).agg(AverageWeekdayVolume = ('TotalCount', 'mean')).reset_index()
max_count = truck_summary['AverageWeekdayVolume'].max()
m2 = folium.Map(location = [32.2226, -110.9747], zoom_start = 10, tiles = 'cartodbpositron')
for index, row in truck_summary.iterrows():
    if row['AverageWeekdayVolume'] > 0:
        folium.CircleMarker(
            location = (row['Latitude'], row['Longitude']),
            radius = calculate_radius(row['AverageWeekdayVolume'], max_count),
            color = '#9a9a9a',
            fill = True,
            fill_opacity = 0.5,
            popup = f"<b>Intersection ID:</b> {row['IntersectionId'].astype(int)}<br>" + 
                    f"<b>Year:</b> {row['Year'].astype(int)}<br>" + 
                    f"<b>Average Weekday Truck Count:</b> {row['AverageWeekdayVolume'].round(2)}"
        ).add_to(m2)
legend_html = '''
<div style = 'position: fixed; bottom: 50px; left: 50px; width: 250px; height: auto; border: 2px solid grey; 
              z-index: 9999; font-size: 14px; background-color: white; padding: 10px;'>
<b>Time</b><br>
AM: 6:30 - 8:30<br>
PM: 4:00 - 6:00<br>
<b>Average Weekday Truck Count</b><br>
'''
for count in [10, 50, 100, 150]:
    radius = calculate_radius(count, max_count) * 1.5
    legend_html += f'''
    <div style = 'margin: 5px 0;'>
        {count}
        <div style = 'display: inline-block; width: {radius:.2f}px; height: {radius:.2f}px; 
                      background-color: #9a9a9a; border-radius: 100%; margin-left: 10px;'></div>
    </div>
    '''
legend_html += '</div>'
m2.get_root().html.add_child(folium.Element(legend_html))
m2.save('truck_peak.html')
print('Map has been created and saved to truck_peak.html')

Map has been created and saved to truck_peak.html


In [ ]:
chunks = []
for chunk in pd.read_csv('D:/Dr. Shamshiripour/PAG_TMC/TmcCountData.CSV', chunksize = 50000,
                         usecols = ['CountId', 'ClassificationId', 'StartTime',
                                    'HardLeftMovement', 'LeftMovement', 'BearLeftMovement', 
                                    'ThruOrPedMovement', 'BearRightMovement', 'RightMovement', 
                                    'HardRightMovement', 'UTurnMovement']):
    chunk['StartTime'] = pd.to_datetime(chunk['StartTime'], format = '%H:%M:%S').dt.time
    chunk = chunk[((chunk['StartTime'] >= am_peak_start) & (chunk['StartTime'] < am_peak_end)) | 
                  ((chunk['StartTime'] >= pm_peak_start) & (chunk['StartTime'] < pm_peak_end))]
    chunk = chunk.merge(count_data, on = 'CountId', how = 'inner')
    chunk = chunk.merge(intersection_data, on = 'IntersectionId', how = 'inner')
    chunk['TotalCount'] = chunk[['HardLeftMovement', 'LeftMovement', 'BearLeftMovement', 
                                 'ThruOrPedMovement', 'BearRightMovement', 'RightMovement', 
                                 'HardRightMovement', 'UTurnMovement']].sum(axis = 1)
    chunk = chunk[chunk['ClassificationId'] == 3]
    chunks.append(chunk)
    del chunk
bikes = pd.concat(chunks, ignore_index = True)
bike_summary = bikes.groupby(['IntersectionId', 'ClassificationId', 'Latitude', 
                              'Longitude', 'StartDate', 'Year'])['TotalCount'].sum().reset_index()
bike_summary = bike_summary.groupby(['IntersectionId', 'ClassificationId', 'Latitude', 
                                     'Longitude', 'Year']).agg(AverageWeekdayVolume = ('TotalCount', 'mean')).reset_index()
max_count = bike_summary['AverageWeekdayVolume'].max()
m3 = folium.Map(location = [32.2226, -110.9747], zoom_start = 10, tiles = 'cartodbpositron')
for index, row in bike_summary.iterrows():
    if row['AverageWeekdayVolume'] > 0:
        folium.CircleMarker(
            location = (row['Latitude'], row['Longitude']),
            radius = calculate_radius(row['AverageWeekdayVolume'], max_count),
            color = '#ff8b8b',
            fill = True,
            fill_opacity = 0.5,
            popup = f"<b>Intersection ID:</b> {row['IntersectionId'].astype(int)}<br>" + 
                    f"<b>Year:</b> {row['Year'].astype(int)}<br>" + 
                    f"<b>Average Weekday Bike Count:</b> {row['AverageWeekdayVolume'].round(2)}"
        ).add_to(m3)
legend_html = '''
<div style = 'position: fixed; bottom: 50px; left: 50px; width: 250px; height: auto; border: 2px solid grey; 
              z-index: 9999; font-size: 14px; background-color: white; padding: 10px;'>
<b>Time</b><br>
AM: 6:30 - 8:30<br>
PM: 4:00 - 6:00<br>
<b>Average Weekday Bike Count</b><br>
'''
for count in [100, 200, 300, 400]:
    radius = calculate_radius(count, max_count) * 1.5
    legend_html += f'''
    <div style = 'margin: 5px 0;'>
        {count}
        <div style = 'display: inline-block; width: {radius:.2f}px; height: {radius:.2f}px; 
                      background-color: #ff8b8b; border-radius: 100%; margin-left: 10px;'></div>
    </div>
    '''
legend_html += '</div>'
m3.get_root().html.add_child(folium.Element(legend_html))
m3.save('bike_peak.html')
print('Map has been created and saved to bike_peak.html')

,IntersectionId,ClassificationId,Latitude,Longitude,Year,AverageWeekdayVolume
0,48604,3,32.507256,-110.925857,2023,7.000000
1,48605,3,32.481702,-110.921876,2023,9.000000
2,48606,3,32.474541,-110.922998,2023,13.000000
3,48607,3,32.440348,-110.933932,2023,9.000000
4,48609,3,32.428238,-111.208426,2022,0.000000
...,...,...,...,...,...,...
608,110089,3,32.380910,-111.029800,2023,3.000000
609,110090,3,32.097630,-110.687590,2023,24.000000
610,111861,3,32.048500,-110.774020,2023,1.105263
611,111862,3,32.132070,-110.734070,2023,10.296512


In [ ]:
chunks = []
for chunk in pd.read_csv('D:/Dr. Shamshiripour/PAG_TMC/TmcCountData.CSV', chunksize = 50000,
                         usecols = ['CountId', 'ClassificationId', 'StartTime',
                                    'HardLeftMovement', 'LeftMovement', 'BearLeftMovement', 
                                    'ThruOrPedMovement', 'BearRightMovement', 'RightMovement', 
                                    'HardRightMovement', 'UTurnMovement']):
    chunk['StartTime'] = pd.to_datetime(chunk['StartTime'], format = '%H:%M:%S').dt.time
    chunk = chunk[((chunk['StartTime'] >= am_peak_start) & (chunk['StartTime'] < am_peak_end)) | 
                  ((chunk['StartTime'] >= pm_peak_start) & (chunk['StartTime'] < pm_peak_end))]
    chunk = chunk.merge(count_data, on = 'CountId', how = 'inner')
    chunk = chunk.merge(intersection_data, on = 'IntersectionId', how = 'inner')
    chunk['TotalCount'] = chunk[['HardLeftMovement', 'LeftMovement', 'BearLeftMovement', 
                                 'ThruOrPedMovement', 'BearRightMovement', 'RightMovement', 
                                 'HardRightMovement', 'UTurnMovement']].sum(axis = 1)
    chunk = chunk[chunk['ClassificationId'] == 4]
    chunks.append(chunk)
    del chunk
pedestrians = pd.concat(chunks, ignore_index = True)
pedestrian_summary = pedestrians.groupby(['IntersectionId', 'ClassificationId', 'Latitude', 
                                          'Longitude', 'StartDate', 'Year'])['TotalCount'].sum().reset_index()
pedestrian_summary = pedestrian_summary.groupby(['IntersectionId', 'ClassificationId', 'Latitude', 
                                     'Longitude', 'Year']).agg(AverageWeekdayVolume = ('TotalCount', 'mean')).reset_index()
max_count = pedestrian_summary['AverageWeekdayVolume'].max()
m4 = folium.Map(location = [32.2226, -110.9747], zoom_start = 10, tiles = 'cartodbpositron')
for index, row in pedestrian_summary.iterrows():
    if row['AverageWeekdayVolume'] > 0:
        folium.CircleMarker(
            location = (row['Latitude'], row['Longitude']),
            radius = calculate_radius(row['AverageWeekdayVolume'], max_count),
            color = '#a7ff94',
            fill = True,
            fill_opacity = 0.5,
            popup = f"<b>Intersection ID:</b> {row['IntersectionId'].astype(int)}<br>" + 
                    f"<b>Year:</b> {row['Year'].astype(int)}<br>" + 
                    f"<b>Average Weekday Pedestrian Count:</b> {row['AverageWeekdayVolume'].round(2)}"
        ).add_to(m4)
legend_html = '''
<div style = 'position: fixed; bottom: 50px; left: 50px; width: 280px; height: auto; border: 2px solid grey; 
              z-index: 9999; font-size: 14px; background-color: white; padding: 10px;'>
<b>Time</b><br>
AM: 6:30 - 8:30<br>
PM: 4:00 - 6:00<br>
<b>Average Weekday Pedestrian Count</b><br>
'''
for count in [400, 800, 1200, 1600]:
    radius = calculate_radius(count, max_count) * 1.5
    legend_html += f'''
    <div style = 'margin: 5px 0;'>
        {count}
        <div style = 'display: inline-block; width: {radius:.2f}px; height: {radius:.2f}px; 
                      background-color: #a7ff94; border-radius: 100%; margin-left: 10px;'></div>
    </div>
    '''
legend_html += '</div>'
m4.get_root().html.add_child(folium.Element(legend_html))
m4.save('pedestrian_peak.html')
print('Map has been created and saved to pedestrian_peak.html')

,IntersectionId,ClassificationId,Latitude,Longitude,Year,AverageWeekdayVolume
0,48604,4,32.507256,-110.925857,2023,9.0
1,48605,4,32.481702,-110.921876,2023,1.0
2,48606,4,32.474541,-110.922998,2023,0.0
3,48607,4,32.440348,-110.933932,2023,1.0
4,48608,4,32.234193,-110.977930,2017,23.0
...,...,...,...,...,...,...
602,65748,4,32.212040,-110.968540,2022,58.0
603,110087,4,32.278360,-110.772500,2023,2.0
604,110088,4,32.268500,-110.787870,2023,0.0
605,110089,4,32.380910,-111.029800,2023,13.0
